In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
Key = os.getenv("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(openai_api_key = Key, model_name = "gpt-3.5-turbo", temperature = 0.5)

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import  get_openai_callback
import PyPDF2


In [6]:


TEMPLATE ="""
Text:{text}
You are an expert in Python programming. Generate {num_questions} multiple-choice questions based on the {topic} provided and answer them using the given {text}.
Each question should have 4 options (A,B,C,D) and one correct answer.
The questions should be of {difficulty} difficulty level.
Generate the questions in the following format:

Question:
(Write the question here)

Options:
A) Option 1
B) Option 2
C) Option 3
D) Option 4

Correct Answer:
(Write the correct option letter here, e.g., “B”)

Explanation:
(Briefly explain why the correct option is right and why others are wrong — optional but recommended.)
##RESPONSE_FORMAT##
{response_json}


##Note: Don't forget to ask the mentioned number of questions in the response.
"""


In [7]:
RESPONSE_JSON = """[
  {
    "Q1": "Which planet is known as the Red Planet?",
    "Options": {
      "A": "Earth",
      "B": "Mars",
      "C": "Jupiter",
      "D": "Venus"
    },
    "Correct Answer": "B) Mars",
    "Explanation": "Mars is called the Red Planet because of its reddish appearance due to iron oxide on its surface."
  },
  {
    "Q2": "What is the chemical symbol for gold?",
    "Options": {
      "A": "Ag",
      "B": "Au",
      "C": "Gd",
      "D": "Go"
    },
    "Correct Answer": "B) Au",
    "Explanation": "The chemical symbol for gold is Au, derived from its Latin name 'Aurum'."
  },
  {
    "Q3": "Which is the largest mammal in the world?",
    "Options": {
      "A": "African Elephant",
      "B": "Blue Whale",
      "C": "Giraffe",
      "D": "Hippopotamus"
    },
    "Correct Answer": "B) Blue Whale",
    "Explanation": "The blue whale is the largest mammal, reaching lengths of up to 100 feet."
  },
  {
    "Q4": "In computing, what does 'HTTP' stand for?",
    "Options": {
      "A": "HyperText Transfer Protocol",
      "B": "High Transfer Text Protocol",
      "C": "Hyper Transfer Text Process",
      "D": "High Transmission Transfer Protocol"
    },
    "Correct Answer": "A) HyperText Transfer Protocol",
    "Explanation": "HTTP stands for HyperText Transfer Protocol, used for transferring web pages on the internet."
  },
  {
    "Q5": "Who developed the theory of relativity?",
    "Options": {
      "A": "Isaac Newton",
      "B": "Albert Einstein",
      "C": "Galileo Galilei",
      "D": "Niels Bohr"
    },
    "Correct Answer": "B) Albert Einstein",
    "Explanation": "Albert Einstein developed the theory of relativity, revolutionizing modern physics."
  }
]
"""

In [8]:
quiz_generation_prompt = PromptTemplate(input_variables=["topic", "difficulty", "num_questions","text","response_json"],
                                        template=TEMPLATE)


In [9]:
chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

C:\Users\user\AppData\Local\Temp\ipykernel_12408\1736729569.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)


In [10]:
TEMPLATE2 = """
You are an expert {topic} reviewer and I need you to {num_questions} and review all of them. Review and analyze the following multiple-choice questions generated by an AI model.
Evaluate the quality of the questions, options, and answers. Provide feedback on clarity, relevance, and correctness from the {quiz}.
"""

In [11]:
quiz_review_prompt = PromptTemplate(input_variables=["quiz","topic"], template=TEMPLATE2)

In [12]:
review_chain = LLMChain(llm=llm, prompt=quiz_review_prompt, output_key="review", verbose=True)

In [13]:
generate_quiz_chain = SequentialChain(
    chains=[chain, review_chain],
    input_variables=["topic", "difficulty", "num_questions", "text","response_json"],
    output_variables=["quiz", "review"],
    verbose=True
    )

In [14]:
file_path = r"C:\Users\user\raghavendra\genai_project\data.txt"


In [15]:
with open(file_path, "r") as file:
    text = file.read()

In [16]:
print(text)

Python is a popular programming language. It was created by Guido van Rossum, and released in 1991.

It is used for:

web development (server-side),
software development,
mathematics,
system scripting.
What can Python do?
Python can be used on a server to create web applications.
Python can be used alongside software to create workflows.
Python can connect to database systems. It can also read and modify files.
Python can be used to handle big data and perform complex mathematics.
Python can be used for rapid prototyping, or for production-ready software development.
Why Python?
Python works on different platforms (Windows, Mac, Linux, Raspberry Pi, etc).
Python has a simple syntax similar to the English language.
Python has syntax that allows developers to write programs with fewer lines than some other programming languages.
Python runs on an interpreter system, meaning that code can be executed as soon as it is written. This means that prototyping can be very quick.
Python can be tr

In [17]:
topic = "Python Programming"
difficulty = "easy"
num_questions = 10

In [18]:
with get_openai_callback() as cb:

    response = generate_quiz_chain(
    {
        "topic":topic,
        "difficulty":difficulty,
        "num_questions":5,
        "text":text,
        "response_json":json.dumps(RESPONSE_JSON)  # Ensure response_json is a valid JSON string
    }
    )
    print("Total Tokens Used:", cb.total_tokens)
    print("Total Cost: $", cb.total_cost)
    
     



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Python is a popular programming language. It was created by Guido van Rossum, and released in 1991.

It is used for:

web development (server-side),
software development,
mathematics,
system scripting.
What can Python do?
Python can be used on a server to create web applications.
Python can be used alongside software to create workflows.
Python can connect to database systems. It can also read and modify files.
Python can be used to handle big data and perform complex mathematics.
Python can be used for rapid prototyping, or for production-ready software development.
Why Python?
Python works on different platforms (Windows, Mac, Linux, Raspberry Pi, etc).
Python has a simple syntax similar to the English language.
Python has syntax that allows developers to write programs with fewer lines than some other programming languages.
Python runs on an interpreter system, meaning that co

C:\Users\user\AppData\Local\Temp\ipykernel_12408\475275216.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_quiz_chain(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert Python Programming reviewer and I need you to 5 and review all of them. Review and analyze the following multiple-choice questions generated by an AI model.
Evaluate the quality of the questions, options, and answers. Provide feedback on clarity, relevance, and correctness from the [
  {
    "Q1": "What is Python primarily used for?",
    "Options": {
      "A": "Graphic design",
      "B": "Web development",
      "C": "Music production",
      "D": "Architectural design"
    },
    "Correct Answer": "B) Web development",
    "Explanation": "Python is primarily used for web development, along with software development, mathematics, and system scripting. It is widely used for creating server-side web applications."
  },
  {
    "Q2": "Which of the following is a key feature of Python syntax?",
    "Options": {
      "A": "Semicolons to end lines",
      "B": "Indentation for defining scope

In [19]:
quiz = response.get("quiz")

In [20]:
print(quiz)

[
  {
    "Q1": "What is Python primarily used for?",
    "Options": {
      "A": "Graphic design",
      "B": "Web development",
      "C": "Music production",
      "D": "Architectural design"
    },
    "Correct Answer": "B) Web development",
    "Explanation": "Python is primarily used for web development, along with software development, mathematics, and system scripting. It is widely used for creating server-side web applications."
  },
  {
    "Q2": "Which of the following is a key feature of Python syntax?",
    "Options": {
      "A": "Semicolons to end lines",
      "B": "Indentation for defining scope",
      "C": "Curly-brackets for loops",
      "D": "Parentheses for functions"
    },
    "Correct Answer": "B) Indentation for defining scope",
    "Explanation": "Python syntax relies on indentation, using whitespace, to define scope such as loops, functions, and classes. This is a key feature that sets Python apart from other programming languages."
  },
  {
    "Q3": "Whic

In [21]:
json_data = json.dumps(RESPONSE_JSON)

In [22]:
json_data 

'"[\\n  {\\n    \\"Q1\\": \\"Which planet is known as the Red Planet?\\",\\n    \\"Options\\": {\\n      \\"A\\": \\"Earth\\",\\n      \\"B\\": \\"Mars\\",\\n      \\"C\\": \\"Jupiter\\",\\n      \\"D\\": \\"Venus\\"\\n    },\\n    \\"Correct Answer\\": \\"B) Mars\\",\\n    \\"Explanation\\": \\"Mars is called the Red Planet because of its reddish appearance due to iron oxide on its surface.\\"\\n  },\\n  {\\n    \\"Q2\\": \\"What is the chemical symbol for gold?\\",\\n    \\"Options\\": {\\n      \\"A\\": \\"Ag\\",\\n      \\"B\\": \\"Au\\",\\n      \\"C\\": \\"Gd\\",\\n      \\"D\\": \\"Go\\"\\n    },\\n    \\"Correct Answer\\": \\"B) Au\\",\\n    \\"Explanation\\": \\"The chemical symbol for gold is Au, derived from its Latin name \'Aurum\'.\\"\\n  },\\n  {\\n    \\"Q3\\": \\"Which is the largest mammal in the world?\\",\\n    \\"Options\\": {\\n      \\"A\\": \\"African Elephant\\",\\n      \\"B\\": \\"Blue Whale\\",\\n      \\"C\\": \\"Giraffe\\",\\n      \\"D\\": \\"Hippopotamus\